# Lesson 2: Tool Calling

## Topic for discussion : Automated Function Execution and Information Retrieval System


This code integrates various tools to perform automated function execution and document information retrieval using the LlamaIndex framework and OpenAI's GPT-3.5-turbo model. Initially, it defines simple function tools (add and mystery) and uses a language model to predict and call these functions based on user input. It then sets up an auto-retrieval tool by loading a document, splitting it into chunks, creating a vector store index, and performing filtered vector queries to retrieve specific information from the document. Additionally, a summary index and a summary query tool are defined to provide comprehensive document summaries, enabling the system to intelligently respond to queries about document content and specific sections.

## Setup

In [1]:
from helper import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()

In [2]:
import nest_asyncio
nest_asyncio.apply()

## 1. Define a Simple Tool

In [8]:
from llama_index.core.tools import FunctionTool

def add(x: int, y: int) -> int:
    """Adds two integers together."""
    return x + y

def mystery(x: int, y: int) -> int: 
    """Mystery function that operates on top of two numbers."""
    return (x + y) * (x + y)


add_tool = FunctionTool.from_defaults(fn=add)
mystery_tool = FunctionTool.from_defaults(fn=mystery)

In [9]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo")
response = llm.predict_and_call(
    [add_tool, mystery_tool], 
    "Tell me the output of the mystery function on 2 and 9", 
    verbose=True
)
print(str(response))

=== Calling Function ===
Calling function: mystery with args: {"x": 2, "y": 9}
=== Function Output ===
121
121


## 2. Define an Auto-Retrieval Tool

### Load Data

Now let us download a dataset and check it out

#!wget "https://openreview.net/pdf?id=VtmBAGCN7o" -O metagpt.pdf


In [11]:
from llama_index.core import SimpleDirectoryReader
# load documents
documents = SimpleDirectoryReader(input_files=["metagpt.pdf"]).load_data()

In [12]:
from llama_index.core.node_parser import SentenceSplitter
splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

### Content of an example chunk

- all meta data would get a whole set of information using the meta data attached

In [13]:
print(nodes[0].get_content(metadata_mode="all"))

page_label: 1
file_name: metagpt.pdf
file_path: metagpt.pdf
file_type: application/pdf
file_size: 16911937
creation_date: 2024-06-16
last_modified_date: 2024-04-23

Preprint
METAGPT: M ETA PROGRAMMING FOR A
MULTI -AGENT COLLABORATIVE FRAMEWORK
Sirui Hong1∗, Mingchen Zhuge2∗, Jonathan Chen1, Xiawu Zheng3, Yuheng Cheng4,
Ceyao Zhang4,Jinlin Wang1,Zili Wang ,Steven Ka Shing Yau5,Zijuan Lin4,
Liyang Zhou6,Chenyu Ran1,Lingfeng Xiao1,7,Chenglin Wu1†,J¨urgen Schmidhuber2,8
1DeepWisdom,2AI Initiative, King Abdullah University of Science and Technology,
3Xiamen University,4The Chinese University of Hong Kong, Shenzhen,
5Nanjing University,6University of Pennsylvania,
7University of California, Berkeley,8The Swiss AI Lab IDSIA/USI/SUPSI
ABSTRACT
Remarkable progress has been made on automated problem solving through so-
cieties of agents based on large language models (LLMs). Existing LLM-based
multi-agent systems can already solve simple dialogue tasks. Solutions to more
complex tasks, however, 

In [14]:
print(nodes[2].get_content(metadata_mode="all"))

page_label: 3
file_name: metagpt.pdf
file_path: metagpt.pdf
file_type: application/pdf
file_size: 16911937
creation_date: 2024-06-16
last_modified_date: 2024-04-23

Preprint
•We introduce MetaGPT, a meta-programming framework for multi-agent collaboration based on
LLMs. It is highly convenient and flexible, with well-defined functions like role definition and
message sharing, making it a useful platform for developing LLM-based multi-agent systems.
•Our innovative integration of human-like SOPs throughout MetaGPT’s design significantly en-
hances its robustness, reducing unproductive collaboration among LLM-based agents. Furthermore,
we introduce a novel executive feedback mechanism that debugs and executes code during runtime,
significantly elevating code generation quality (e.g., 5.4% absolute improvement on MBPP).
•We achieve state-of-the-art performance on HumanEval (Chen et al., 2021a) and MBPP (Austin
et al., 2021). Extensive results convincingly validate MetaGPT, suggesting that

### 

In [15]:
from llama_index.core import VectorStoreIndex

vector_index = VectorStoreIndex(nodes)
query_engine = vector_index.as_query_engine(similarity_top_k=2)

In [17]:
from llama_index.core.vector_stores import MetadataFilters

query_engine = vector_index.as_query_engine(
    similarity_top_k=2,
    filters=MetadataFilters.from_dicts(
        [
            {"key": "page_label", "value": "2"}
        ]
    )
)

response = query_engine.query(
    "What are some high-level results of MetaGPT?", 
)

In [18]:
print(str(response))

Some high-level results of MetaGPT include achieving a new state-of-the-art in code generation benchmarks with high Pass@1 scores, outperforming other popular frameworks in creating complex software projects, handling higher levels of software complexity effectively, offering extensive functionality, and demonstrating robustness and efficiency in task completion rates.


In [19]:
for n in response.source_nodes:
    print(n.metadata)

{'page_label': '2', 'file_name': 'metagpt.pdf', 'file_path': 'metagpt.pdf', 'file_type': 'application/pdf', 'file_size': 16911937, 'creation_date': '2024-06-16', 'last_modified_date': '2024-04-23'}


### Define the Auto-Retrieval Tool

LLM can integently infer relevant meta data filters based on the user query

Meta data filter like section ids 

In [20]:
from typing import List
from llama_index.core.vector_stores import FilterCondition


def vector_query(
    query: str, 
    page_numbers: List[str]
) -> str:
    """Perform a vector search over an index.
    
    query (str): the string query to be embedded.
    page_numbers (List[str]): Filter by set of pages. Leave BLANK if we want to perform a vector search
        over all pages. Otherwise, filter by the set of specified pages.
    
    """

    metadata_dicts = [
        {"key": "page_label", "value": p} for p in page_numbers
    ]
    
    query_engine = vector_index.as_query_engine(
        similarity_top_k=2,
        filters=MetadataFilters.from_dicts(
            metadata_dicts,
            condition=FilterCondition.OR
        )
    )
    response = query_engine.query(query)
    return response
    

vector_query_tool = FunctionTool.from_defaults(
    name="vector_tool",
    fn=vector_query
)

In [21]:
llm = OpenAI(model="gpt-3.5-turbo", temperature=0)
response = llm.predict_and_call(
    [vector_query_tool], 
    "What are the high-level results of MetaGPT as described on page 2?", 
    verbose=True
)

=== Calling Function ===
Calling function: vector_tool with args: {"query": "high-level results of MetaGPT", "page_numbers": ["2"]}
=== Function Output ===
MetaGPT achieves a new state-of-the-art (SoTA) in code generation benchmarks with 85.9% and 87.7% in Pass@1. It stands out in handling higher levels of software complexity and offering extensive functionality, demonstrating a 100% task completion rate in experimental evaluations.


In [22]:
for n in response.source_nodes:
    print(n.metadata)

{'page_label': '2', 'file_name': 'metagpt.pdf', 'file_path': 'metagpt.pdf', 'file_type': 'application/pdf', 'file_size': 16911937, 'creation_date': '2024-06-16', 'last_modified_date': '2024-04-23'}


## Let's add some other tools!

In [23]:
from llama_index.core import SummaryIndex
from llama_index.core.tools import QueryEngineTool

summary_index = SummaryIndex(nodes)
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
summary_tool = QueryEngineTool.from_defaults(
    name="summary_tool",
    query_engine=summary_query_engine,
    description=(
        "Useful if you want to get a summary of MetaGPT"
    ),
)

In [24]:
response = llm.predict_and_call(
    [vector_query_tool, summary_tool], 
    "What are the MetaGPT comparisons with ChatDev described on page 8?", 
    verbose=True
)

=== Calling Function ===
Calling function: vector_tool with args: {"query": "MetaGPT comparisons with ChatDev", "page_numbers": ["8"]}
=== Function Output ===
MetaGPT outperforms ChatDev on the SoftwareDev dataset in various metrics. For instance, MetaGPT achieves a higher score in executability, takes less time for execution, uses more tokens but requires fewer tokens to generate a line of code compared to ChatDev. Additionally, MetaGPT shows better performance in code statistics and human revision cost when compared to ChatDev.


In [25]:
for n in response.source_nodes:
    print(n.metadata)

{'page_label': '8', 'file_name': 'metagpt.pdf', 'file_path': 'metagpt.pdf', 'file_type': 'application/pdf', 'file_size': 16911937, 'creation_date': '2024-06-16', 'last_modified_date': '2024-04-23'}


In [26]:
response = llm.predict_and_call(
    [vector_query_tool, summary_tool], 
    "What is a summary of the paper?", 
    verbose=True
)

=== Calling Function ===
Calling function: summary_tool with args: {"input": "Get a summary of the paper."}
=== Function Output ===
The paper introduces MetaGPT, a meta-programming framework for multi-agent collaboration based on Large Language Models (LLMs). MetaGPT incorporates Standardized Operating Procedures (SOPs) to enhance collaboration efficiency and reduce errors among agents. It utilizes role specialization, structured communication interfaces, and executable feedback mechanisms to streamline workflows. Through extensive experiments and evaluations on various datasets, MetaGPT demonstrates superior performance in code generation benchmarks, outperforming existing frameworks. The framework is designed to enhance problem-solving capabilities in multi-agent systems by modeling agents as a simulated software company, emphasizing role specialization, workflow management, and efficient communication mechanisms. MetaGPT also introduces the concept of AgentStore for managing agents 